# OpenAPI to Pydantic-AI Tools - Complete Demo

This notebook demonstrates the complete workflow for converting OpenAPI specifications into pydantic-ai tools.

In [1]:
# Step 1: Load the OpenAPI Tools

import sys
from pathlib import Path

from meta_ally.lib.openapi_to_tools import OpenAPIToolsLoader

# Add project root to Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Initialize the loader
loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json"
)

# Load all tools
print("Loading tools from OpenAPI spec...")
tools = loader.load_tools()

print(f"\n✅ Successfully loaded {len(tools)} tools!")
print("\nFirst 5 tools:")
for i, tool in enumerate(tools[:5], 1):
    print(f"  {i}. {tool.name}")

Loading tools from OpenAPI spec...
Generating models file 'api_models.py' from https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json...
Successfully generated 'api_models.py'
Created tool: get_portal_config [GET /api/info/config]
Created tool: list_models [GET /api/info/models]
Created tool: list_scopes [GET /api/info/scopes]
Created tool: list_copilots [GET /api/copilots]
Created tool: create_copilot [POST /api/copilots]
Created tool: delete_copilot [DELETE /api/copilots]
Created tool: get_copilot_metadata [GET /api/copilots/metadata]
Created tool: update_copilot_metadata [PUT /api/copilots/metadata]
Created tool: get_copilot_config [GET /api/copilots/config]
Created tool: update_copilot_config [POST /api/copilots/config]
Created tool: validate_copilot_config [POST /api/copilots/config/validate]
Created tool: get_copilot_config_history [GET /api/copilots/config/history]
Created tool: list_endpoint_data [GET /api/copilots/data]
Created tool: download_endpoint_data [GET /a

In [2]:
# Check out Tool Definitions
for tool in tools:
    print(f"{tool.tool_def}")

ToolDefinition(name='get_portal_config', parameters_json_schema={'type': 'object', 'properties': {}, 'required': [], 'additionalProperties': False}, description='Retrieve the current configuration parameters of the Ally Portal.\n\nThis API endpoint provides essential capability details required for\nmanaging Copilot configurations within the Ally Portal. It returns vital\ninformation including the Keycloak authentication server URL, security\nrealm, client identifier, as well as the Ally server domain and AI knowledge\nhost URL.\n\nRequired Permissions:\n    None - this endpoint is publicly accessible.\n\nReturns:\n    CapabilitiesResponse: An object containing the following configuration\n    parameters:\n        - Keycloak server URL: The base URL for the Keycloak authentication\n          server.\n        - Realm: The security realm used within Keycloak.\n        - Client ID: The client identifier registered in Keycloak.\n        - Ally server domain: The domain address of the Ally 

In [5]:
# Get specific tool definitions
special_tool = loader.get_tool_by_operation_id("get_copilot_ratings")
if special_tool is not None:
    print(special_tool.tool_def)

special_tool = loader.get_tool_by_operation_id("get_copilot_sessions")
if special_tool is not None:
    print(special_tool.tool_def)


special_tool = loader.get_tool_by_operation_id("get_copilot_session")
if special_tool is not None:
    print(special_tool.tool_def)


special_tool = loader.get_tool_by_operation_id("get_copilot_sessions_summaries")
if special_tool is not None:
    print(special_tool.tool_def)

special_tool = loader.get_tool_by_operation_id("get_copilot_cost_daily")
if special_tool is not None:
    print(special_tool.tool_def)

ToolDefinition(name='get_copilot_ratings', parameters_json_schema={'type': 'object', 'properties': {'endpoint': {'type': 'string', 'description': 'The endpoint identifier for the Copilot (format: /SCOPE/NAME)'}, 'start_time': {'type': 'string', 'description': 'The start datetime to retrieve ratings from'}, 'end_time': {'type': 'string', 'description': 'The end datetime to retrieve ratings up to'}}, 'required': ['endpoint'], 'additionalProperties': False}, description='Retrieve ratings data from AWS DynamoDB within a specified time range.\n\nThis method fetches ratings stored in AWS DynamoDB, optionally filtered by a\ngiven time interval defined by start and end timestamps. It can target a\nspecific Copilot if provided. Requires USE_ALLY permission.\n\nRequired Permissions:\n    `browse` global permission.\n\nArgs:\n    userauth (UserAuthorization):\n        User authorization object containing user permissions and\n        authentication details.\n    endpoint:\n        The endpoint of

In [ ]:
# Step 2: Organize Tools into Groups (Auto-Updated)

# Organize tools by functionality with automatic categorization
tool_groups = {
    "portal_info": [],           # Portal configuration and capabilities
    "copilot": [],               # All copilot operations (CRUD, config, metadata, auth)
    "evaluation": [],            # Test suites and evaluation execution
    "analytics": [],             # Logs, ratings, costs, sessions
    "permissions": [],           # Role-based access control
    "file_operations": [],       # File uploads
    "other": []
}

# Define keyword-based categorization rules with priority order
categorization_rules = [
    # Portal info - highest priority
    ("portal_info", ["get_portal_config", "list_models", "list_scopes"]),

    # Copilot operations (management, metadata, config, authorization)
    ("copilot", [
        # Management
        "list_copilots", "create_copilot", "delete_copilot",
        # Metadata
        "get_copilot_metadata", "update_copilot_metadata",
        # Configuration
        "get_copilot_config", "update_copilot_config", "validate_copilot_config", "get_copilot_config_history",
        # Authorization
        "get_copilot_authorization", "update_copilot_authorization", "delete_copilot_authorization"
    ]),

    # Evaluation (suites management + execution)
    ("evaluation", [
        "list_copilot_evaluation_suites", "get_copilot_evaluation_suite",
        "create_copilot_evaluation_suite", "update_copilot_evaluation_suite",
        "get_copilot_evaluation_suite_history", "add_copilot_evaluation_test_cases",
        "execute_copilot_evaluation_suite", "get_copilot_evaluation_results"
    ]),

    # Analytics: logs, costs, ratings, sessions
    ("analytics", [
        "get_copilot_logs",
        "get_copilot_cost_graph", "get_copilot_cost_daily",
        "get_copilot_ratings",
        "get_copilot_sessions", "get_copilot_sessions_summaries", "get_copilot_session"
    ]),

    # Permissions (role-based access control)
    ("permissions", [
        "get_permissions", "add_role", "remove_role",
        "grant_permission", "revoke_permission", "add_user", "remove_user"
    ]),

    # File operations
    ("file_operations", ["upload_file_to_s3"]),
]

# Categorize each tool based on exact name matching first, then keywords
for tool in tools:
    categorized = False

    # Check exact name matches and keyword matches
    for category, identifiers in categorization_rules:
        if tool.name in identifiers:
            tool_groups[category].append(tool)
            categorized = True
            break
        elif any(identifier in tool.name.lower() for identifier in identifiers if len(identifier) > 3):
            tool_groups[category].append(tool)
            categorized = True
            break

    # If no category matched, put in "other"
    if not categorized:
        tool_groups["other"].append(tool)

# Display the groups and their tools
print("\n📊 Tool Groups Summary:\n")
print("=" * 80)
total_tools = 0
for group_name, group_tools in tool_groups.items():
    if group_tools:  # Only show non-empty groups
        print(f"\n{group_name.upper().replace('_', ' ')}: {len(group_tools)} tools")
        print("-" * 80)
        for tool in group_tools:
            # Truncate description for readability
            desc = tool.description.split('\n')[0][:100]
            print(f"  • {tool.name}")
            print(f"    {desc}...")
        total_tools += len(group_tools)

print("\n" + "=" * 80)
print(f"✅ Total tools categorized: {total_tools}/{len(tools)}")

# Show summary statistics
print("\n📈 Category Distribution:")
print("-" * 80)
for group_name, group_tools in sorted(tool_groups.items(), key=lambda x: len(x[1]), reverse=True):
    if group_tools:
        percentage = (len(group_tools) / len(tools)) * 100
        bar = "█" * int(percentage / 2)  # Visual bar chart
        print(f"  {group_name:.<25} {len(group_tools):>3} ({percentage:>5.1f}%) {bar}")


📊 Tool Groups Summary:


PORTAL INFO: 3 tools
--------------------------------------------------------------------------------
  • get_portal_config
    Retrieve the current configuration parameters of the Ally Portal....
  • list_models
    Retrieve the list of AI models from the model registry....
  • list_scopes
    Retrieve all business unit scopes available in the system....

COPILOT: 13 tools
--------------------------------------------------------------------------------
  • list_copilots
    Retrieve a list of Copilots filtered by an optional prefix....
  • create_copilot
    Create a new Copilot with specified attributes and metadata....
  • delete_copilot
    Delete an existing Copilot and all associated data....
  • get_copilot_metadata
    Retrieve metadata information for a specified Copilot....
  • update_copilot_metadata
    Update metadata for a specified Copilot....
  • get_copilot_config
    Retrieve the active or a historical configuration for a specified Copilot...